In [ ]:
# %matplotlib widget
import scipp as sc
import scippneutron as scn
import plopp as pp
from ess import reflectometry
from datetime import datetime
from data import get_direct_beam, get_sample, get_mantid
import platform
import numpy as np

import ess
from scitacean.testing.client import FakeClient
from scitacean.testing.transfer import FakeFileTransfer

from orsopy import fileio

pp.patch_scipp()

In [ ]:
logger = ess.logging.configure_workflow('offspec_reduction',
                                        filename='offspec.log')

# Set up the SciCat database locally

First, we will set up a fake SciCat database and add our raw data to this. 
This is achieved with the `scitacean` package, which is also compatible with the real ESS SciCat instance. 
The direct beam and sample (which is data from a system of quartz and copper at the air-silicon interface) NeXus files are added to a dataset that is "uploaded" to the fake SciCat instance. 

In [ ]:
import scitacean
scitacean.__version__

In [ ]:
client = FakeClient(file_transfer=FakeFileTransfer(source_folder='/'))

direct_beam_uploaded = get_direct_beam(client)
sample_uploaded = get_sample(client, direct_beam_uploaded)

## Loading the data

The data can then be obtained from the SciCat instance and loaded below. 
The direct beam measurement is obtained by positioning the detector directly in the beam of incident neutrons and moving the sample out of the way and gives an estimate for the ISIS pulse structure as a function of time-of-flight. 

In [ ]:
sample_remote = client.get_dataset(sample_uploaded.pid)
sample_local = client.download_files(sample_remote, target=".")
sample = scn.load(sample_local.files[0].local_path, mantid_args={"LoadMonitors": True})
sample.coords['theta'] = sample.attrs.pop('Theta').value[-1].data

In [ ]:
direct_beam_remote = client.get_dataset(direct_beam_uploaded.pid)
direct_beam_local = client.download_files(direct_beam_remote, target=".")
direct_beam = scn.load(direct_beam_local.files[0].local_path, mantid_args={"LoadMonitors": True})

## Populating the ORSO header

We will write the reduced data file following the [ORSO `.ort` standard](https://www.reflectometry.org/file_format/specification), to enable a metadata rich header. 
We will create an empty header and then populate this. 

### The data source information

Many aspects of the ORSO header can be populate directly from the raw data file (in particular within the `data_source` section). 

In [ ]:
header = fileio.orso.Orso.empty()

header.data_source.owner = fileio.base.Person(name=sample_local.owner, 
                                              affiliation='ISIS Neutron and Muon Source', 
                                              contact=sample_local.contact_email)
header.data_source.experiment = fileio.data_source.Experiment(title=sample_local.name, 
                                                              instrument=sample_local.instrument_id, 
                                                              start_date=sample_local.end_time, 
                                                              probe='neutron', 
                                                              facility=sample_local.creation_location)
header.data_source.sample = fileio.data_source.Sample(name=sample_local.meta['sample_name'], 
                                                      category=sample_local.meta['sample_category'], 
                                                      composition=sample_local.meta['sample_composition'])
header.data_source.measurement = fileio.data_source.Measurement(instrument_settings=fileio.data_source.InstrumentSettings(incident_angle=fileio.base.Value(sample.coords['theta'].value, sample.coords['theta'].unit), 
                                                                                                                          wavelength=None,
                                                                                                                          polarization='unpolarized'),
                                                                data_files=[str(sample_local.pid), str(direct_beam_local.pid)],
                                                                scheme='energy-dispersive')

### The reduction details

The `reduction` section can start to be populated also. 
Entries such as `corrections` will be filled up through the reduction process. 

In [ ]:
header.reduction.software = fileio.reduction.Software(name='ess', version=ess.__version__, platform=platform.platform())
header.reduction.timestamp = datetime.now()
header.reduction.creator = fileio.base.Person(name='Andrew R. McCluskey', 
                                              affiliation='European Spallation Source', 
                                              contact='andrew.mccluskey@ess.eu')
header.reduction.corrections = []
header.reduction.computer = platform.node()
header.reduction.script = '/Users/andrewmccluskey/work/reduction/ess/docs/instruments/external/offspec/offspec_mantid.ipynb'

To ensure that the header object is carried through the process, we assign it to the sample `scipp.DataArray`. 
The direct beam header object will be overwritten at the normalisation step so we will keep this empty. 

In [ ]:
sample.attrs['orso'] = sc.scalar(header)
direct_beam.attrs['orso'] = sc.scalar(fileio.orso.Orso.empty())

### Correcting the position of detector pixels

The pixel positions in sample file must be modified to account for the transformation on the detector by rotating it around the sample. 
We can achieve this by understanding that the sample has been rotated by some amount (as discussed above) and that sample measurement has the specular peak at the same pixel as the direct beam measurement has the direct beam. 
Therefore, we move the sample detector along the arc of the sample rotation by $2\omega$ (in the OFFSPEC files, $\omega$ is called `'Theta'`, which we stored as `'theta'` earlier). 

In [ ]:
from scipp.spatial import rotations_from_rotvecs

def pixel_position_correction(data: sc.DataArray):
    rotation = rotations_from_rotvecs(rotation_vectors=sc.vector(value=[-2.0 * data.coords['theta'].value, 0, 0], unit=sc.units.deg))
    return rotation * (data.coords['position'] - data.coords['sample_position'])

In [ ]:
logger.info("Correcting pixel positions in 'sample.nxs'")
sample.coords['position'] = pixel_position_correction(sample)
sample.attrs['orso'].value.data_source.measurement.comment = 'Pixel positions corrected'

We can visualize the data with a plot. 
In this plot of `sample`, we can see the specular intensity at around spectrum numbers 400-410. 
There is a more intense region, closer to spectrum number 300, which comes from the direct beam of neutrons traveling straight through our sample. 

In [ ]:
sample.hist(tof=50).plot(norm='log')

A region of interest is then defined for the detector. 
This is defined as twenty-five pixels around the specular peak or the direct beam. 
The `scipp.DataArray` is concatenated along the `'spectrum'` coordinate at this stage, essentially collaposing all of the events onto a single pixel.

In [ ]:
sample_roi = sample['spectrum', 389:415].bins.concat('spectrum')
direct_beam_roi = direct_beam['spectrum', 389:415].bins.concat('spectrum')

sample_roi.attrs['orso'].value.reduction.corrections += ['region of interest defined as spectrum 389:415']

The position of these events is then defined as the centre of the region of interest. 

In [ ]:
sample_roi.coords['position'] = sample.coords['position'][401]
direct_beam_roi.coords['position'] = direct_beam.coords['position'][401]

## Coordinate transform graph

To compute the wavelength $\lambda$, the scattering angle $\theta$, and the $Q$ vector for our data we can use a coordinate transform graph. 
The reflectometry graph is discussed in detail in the [Amor reduction notebook](https://scipp.github.io/ess/instruments/amor/amor_reduction.html) and the one used here is nearly identical.
The only difference is the Amor instrument uses choppers to define the pulse of neutrons, which is not the case here. 
The OFFSPEC graph is the standard reflectometry graph, shown below. 

In [ ]:
graph = {**reflectometry.conversions.specular_reflection()}
sc.show_graph(graph, simplified=True)

## Computing the wavelength

The neutron wavelengths can be computed with `transform_coords` and the graph shown above. 
We will only use neutrons in the wavelength range of 2 Å to 15.0 Å. 

In [ ]:
wavelength_edges = sc.linspace(dim='wavelength', start=2, stop=15, num=2, unit='angstrom')
sample_wav = reflectometry.conversions.tof_to_wavelength(sample_roi, wavelength_edges,graph=graph)

Since the direct beam measurement is **not** a reflectometry measurement, we use the `no_scatter_graph` to convert this to wavelength.

In [ ]:
no_scatter_graph = {**scn.conversion.graph.beamline.beamline(scatter=False),
                    **scn.conversion.graph.tof.elastic_wavelength(start='tof')}
sc.show_graph(no_scatter_graph, simplified=True)

direct_beam_wav = direct_beam_roi.transform_coords('wavelength', graph=no_scatter_graph)
direct_beam_wav = direct_beam_wav.bin(wavelength=wavelength_edges)

## Normalization by monitor

It is necessary to normalize the sample and direct beam measurements by the summed monitor counts, which accounts for different lengths of measurement and long-timescale natural variation in the pulse. 
This will ensure that the final data has the correct scaling when the reflectivity data is normalized.
First, we convert the data to wavelength, using the `no_scatter_graph` used previously for the direct beam.

The most reliable monitor for the OFFSPEC instrument is `'monitor2'` in the file, therefore this is used. 

In [ ]:
sample_mon_wav = sample_roi.attrs['monitor2'].value.transform_coords('wavelength', graph=no_scatter_graph)
direct_beam_mon_wav = direct_beam_roi.attrs['monitor2'].value.transform_coords('wavelength', graph=no_scatter_graph)

A background subtraction is then performed on the monitor data, where the background as taken as any counts at wavelengths greater than 15 Å. 

In [ ]:
sample_mon_wav -= sc.values(sample_mon_wav['wavelength', 15 * sc.units.angstrom].mean())
direct_beam_mon_wav -= sc.values(direct_beam_mon_wav['wavelength', 15 * sc.units.angstrom].mean())
sample_wav.attrs['orso'].value.reduction.corrections += ['monitor background subtraction, background above 15 Å']

The monitors are then summed along the `'wavelength'` and this value is used to normalise the data. 

In [ ]:
sample_mon_wav_sum = sample_mon_wav['wavelength', 2 * sc.units.angstrom:15 * sc.units.angstrom].sum()
direct_beam_mon_wav_sum = direct_beam_mon_wav['wavelength', 2 * sc.units.angstrom:15 * sc.units.angstrom].sum()
sample_norm = sample_wav / sc.values(sample_mon_wav_sum)
direct_beam_norm = direct_beam_wav / sc.values(direct_beam_mon_wav_sum)
sample_wav.attrs['orso'].value.reduction.corrections += ['normalisation by summed monitor']

## Normalisation of sample by direct beam

The sample and direct beam measurements (which have been normalised by monitor counts) are then histogrammed in wavelength to 100 geometrically spaced points. 
The histogrammed direct beam is then used to normalised the sample. 

Importantly, some relevant metadata (including the ORSO header object) is carried over. 

In [ ]:
sample_norm_hist = sample_norm.hist(wavelength=sc.geomspace(dim='wavelength', start=2, stop=14, num=100, unit=sc.units.angstrom))
direct_beam_norm_hist = direct_beam_norm.hist(wavelength=sc.geomspace(dim='wavelength', start=2, stop=14, num=100, unit=sc.units.angstrom))
sample_norm_hist.coords['sample_position'] = sample_norm.attrs.pop('sample_position')

norm_wav = sample_norm_hist / direct_beam_norm_hist
norm_wav.attrs['orso'] = sample_wav.attrs['orso']
norm_wav.coords['theta'] = sample_wav.coords['theta']

norm_wav.attrs['orso'].value.reduction.corrections += ['normalisation by direct beam']

## Conversion to $Q$

This normalised data can then be used to compute the reflectivity as a function of the scattering vector $Q$. 

In [ ]:
norm_q = reflectometry.conversions.theta_to_q(norm_wav, graph=graph)

Which we can visualise.

In [ ]:
norm_q.plot(norm='log')

## Comparison with Mantid

The same data was reduced with Mantid for comparison. 

We will add this to our SciCat instance as a derived dataset of the sample. 

In [ ]:
reduced_mantid = sample_local.derive()
mantid_file = get_mantid()
reduced_mantid.add_local_files(str(mantid_file), base_path=str(mantid_file.parents[0]))
reduced_mantid.contact_email = 'jos.cooper@stfc.ac.uk'
reduced_mantid.name = 'Reduced (by mantid) reflectometry data from QCS sample'
reduced_mantid.used_software = ['mantid']
reduced_mantid.investigator = 'Joshanial F. K. Cooper'
reduced_mantid.owner_group = sample_local.owner_group
reduced_mantid.input_datasets.append(direct_beam_local.pid)

reduced_mantid_up = client.upload_new_dataset_now(reduced_mantid)

We can then get this dataset and load it (with `numpy` as it is not an ORSO file) before transforming it to a `scipp.DataArray` for each plotting.

In [ ]:
reduced_mantid_remote = client.get_dataset(reduced_mantid_up.pid)
reduced_mantid_local = client.download_files(reduced_mantid_remote, target=".")
mantid_q, mantid_r, mantid_dr = np.loadtxt(reduced_mantid_local.files[0].local_path, delimiter=',', unpack=True)

mantid = sc.DataArray(coords={'Q': sc.array(dims=['Q'], values=mantid_q, unit='1/Å')}, 
                      data=sc.array(dims=['Q'], values=mantid_r, variances=np.square(mantid_dr)))

We can see there is excellent agreement between the two pieces of software. 
Small differences present are a result of the different $Q$-bins that are used. 

In [ ]:
sc.plot({'Mantid': mantid, 'scipp': (norm_q)}, norm='log')

## Saving the scipp-reduced data as .ort

We constructed the ORSO header through the reduction process. 
We can now make use of this when we save our .ort file. 

First, we will assume a 3 % of $Q$ resolution function to be included in our file.

In [ ]:
norm_q.coords['sigma_Q'] = sc.midpoints(norm_q.coords['Q']) * 0.03

Then, due a [bug in orsopy](https://github.com/reflectivity/orsopy/pull/101), we need to overwrite the incident angle and wavelength that have been out-populated by the reduction. 

In [ ]:
incident_angle = norm_q.attrs['orso'].value.data_source.measurement.instrument_settings.incident_angle
wavelength = norm_q.attrs['orso'].value.data_source.measurement.instrument_settings.wavelength

norm_q.attrs['orso'].value.data_source.measurement.instrument_settings.wavelength = fileio.base.ValueRange(min=float(wavelength.min), max=float(wavelength.max), unit=wavelength.unit)
norm_q.attrs['orso'].value.data_source.measurement.instrument_settings.incident_angle = fileio.base.Value(magnitude=float(incident_angle.magnitude), unit=incident_angle.unit)

And it is necessary to add the column for our uncertainties, which details the **meaning** of the uncertainty values we have given. 

In [ ]:
norm_q.attrs['orso'].value.columns.append(fileio.base.ErrorColumn(error_of='R', error_type='uncertainty', value_is='sigma'))
norm_q.attrs['orso'].value.columns.append(fileio.base.ErrorColumn(error_of='Q', error_type='resolution', value_is='sigma'))

Finally, we can save the file.

In [ ]:
reflectometry.io.save_ort(norm_q, 'offspec.ort')

In [ ]:
!head offspec.ort

## Adding the reduced data to SciCat

The final step is to upload out new reduced data to SciCat as a derived dataset.

In [ ]:
reduced = sample_local.derive()

reduced.add_local_files('./offspec.ort', base_path='.')
reduced.contact_email = 'andrew.mccluskey@ess.eu'
reduced.input_datasets.append(direct_beam_local.pid)
reduced.name = 'Reduced reflectometry data from QCS sample'
reduced.used_software = ['scipp-23.01.1']
reduced.investigator = 'Andrew R. McCluskey'
reduced.owner_group = sample_local.owner_group

reduced_up = client.upload_new_dataset_now(reduced)